In [ ]:
import pandas as pd
import numpy as np
import nltk

In [ ]:
platforms = ["Pitchfork", "Guardian", "NME", "Spectrum"]
dfs = {platform: pd.read_hdf(
    "C:\\Users\\tommy\\OneDrive\\University\\Year 3\\Third Year Project\\Platform Album Data\\text_edited_data.h5", key=platform).drop(
    ["Url"], axis=1) for platform in platforms}

text = {platform: df.loc[:, "Text"] for platform, df in dfs.items()}

In [ ]:
from nltk.tokenize.nist import NISTTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string

nist_tokenizer = NISTTokenizer()
data = dfs["Pitchfork"].loc[0, "Text"]
test_text = data

In [ ]:
import re

def tokenize_both(x):
    
    return [nist_tokenizer.tokenize(sentence) for sentence in sent_tokenize(x)]

def replace_quotes(x):
    
    for match in re.findall(" “.+?[”'\1{2}]", x):
        x = x.replace(match, "")
                
    return x

def preprocess_unigram(x):
    
    x = replace_quotes(x)
    x = tokenize_both(x)
    x = [[word for word in sentence if word[0] not in string.punctuation] for sentence in x]
    x = [nltk.pos_tag(text) for text in x]
    x = np.array([np.array([word for word in sentence if word[0].lower() not in stopwords.words("english")]) for sentence in x])
    
    return x

def preprocess_bigram(x):
    
    convert_tuple = [[list(word) for word in sentence] for sentence in preprocess_unigram(x)]
    
    return [list(nltk.bigrams(sentence)) for sentence in convert_tuple]

test_text = preprocess(test_text)
test_text

In [ ]:
from nltk.corpus import opinion_lexicon

ops_count = np.array([sum(list(map(lambda x: x[0].lower() in opinion_lexicon.words(), sentence))) / len(sentence) for sentence in test_text])

In [ ]:
ops_count

In [ ]:
test_text[ops_count <= 0.1]

In [ ]:
noun_count = np.array([sum(list(map(lambda x: x[1] in ["NNP", "NN"], sentence))) / len(sentence) for sentence in test_text])
noun_count

In [ ]:
bigrams = [list(nltk.bigrams(sentence)) for sentence in 
           [[list(word) for word in sentence] for sentence in preprocess(data)]]
bigrams

In [ ]:
nn_count = np.array([sum(list(map(lambda x: x[0][1] in ["NNP", "NN"] and x[1][1] in ["NNP", "NN"], sentence))) for sentence in bigrams])
an_count = np.array([sum(list(map(lambda x: x[0][1] == "JJ" and x[1][1] in ["NNP", "NN"], sentence))) for sentence in bigrams])

print(nn_count, an_count)